In [2]:

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier



In [3]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [4]:
import pickle
import json
import pandas as pd

from utils.dataloader import DataLoader 
from settings.constants import TRAIN_CSV


with open('settings/specifications.json') as f:
    specifications = json.load(f)

raw_train = pd.read_csv(TRAIN_CSV)
x_columns = specifications['description']['X']
y_column = specifications['description']['y']

x_raw = raw_train[x_columns]

loader = DataLoader()
loader.fit(x_raw)
X = loader.load_data()
y = raw_train['target']

model = KNeighborsClassifier(n_neighbors=6)
model.fit(X, y)
with open('models/KNN.pickle', 'wb')as f:
    pickle.dump(model, f)

In [5]:
raw_train

,Vmag,Plx,e_Plx,B-V,Amag,target
0,6.58,2.83,0.52,-0.159,13.838932,0
1,8.78,1.49,1.22,1.194,14.645931,0
2,8.30,1.85,1.06,1.071,14.635859,0
3,8.44,2.45,0.99,0.139,15.385830,1
4,8.49,3.58,0.84,1.261,16.259415,0
...,...,...,...,...,...,...
14735,7.52,13.85,0.60,0.584,18.227250,1
14736,7.38,4.87,0.85,1.036,15.817645,0
14737,8.96,7.74,1.07,0.576,18.403706,1
14738,8.84,4.47,1.14,0.972,17.091537,0


In [6]:
import pickle
import json
import pandas as pd

from utils.dataloader import DataLoader 
from settings. constants import VAL_CSV


with open('settings/specifications.json') as f:
    specifications = json.load(f)

x_columns = specifications['description']['X']
y_column = specifications['description']['y']

raw_val = pd.read_csv(VAL_CSV)
x_raw = raw_val[x_columns]

loader = DataLoader()
loader.fit(x_raw)
X = loader.load_data()
y = raw_val.target

loaded_model = pickle.load(open('models/KNN.pickle', 'rb'))
loaded_model.score(X, y)

0.8686131386861314

In [7]:
X

,Vmag,Plx,e_Plx,B-V,Amag
0,7.83,4.88,1.22,1.015,16.272099
1,10.69,0.73,1.28,-0.077,15.006614
2,7.02,9.23,0.85,0.518,16.846008
3,6.43,8.89,0.76,0.992,16.174510
4,8.47,11.01,0.94,0.549,18.678936
...,...,...,...,...,...
7256,7.95,7.60,1.07,0.272,17.354069
7257,6.78,16.08,0.94,0.954,17.811430
7258,9.53,8.22,1.35,0.537,19.104359
7259,9.44,29.37,1.46,1.029,21.779520


In [8]:
X

,Vmag,Plx,e_Plx,B-V,Amag
0,7.83,4.88,1.22,1.015,16.272099
1,10.69,0.73,1.28,-0.077,15.006614
2,7.02,9.23,0.85,0.518,16.846008
3,6.43,8.89,0.76,0.992,16.174510
4,8.47,11.01,0.94,0.549,18.678936
...,...,...,...,...,...
7256,7.95,7.60,1.07,0.272,17.354069
7257,6.78,16.08,0.94,0.954,17.811430
7258,9.53,8.22,1.35,0.537,19.104359
7259,9.44,29.37,1.46,1.029,21.779520


In [10]:
import json
import requests
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from utils import DataLoader, Estimator 
from settings. constants import TRAIN_CSV, VAL_CSV

with open('settings/specifications.json') as f:
    specifications = json.load(f)
    
info = specifications['description']
x_columns, y_column, metrics = info['X'], info['y'], info['metrics']

train_set = pd.read_csv(TRAIN_CSV, header=0)
val_set = pd.read_csv(VAL_CSV, header=0)

train_x, train_y = train_set[x_columns], train_set[y_column]
val_x, val_y = val_set[x_columns], val_set[y_column]

loader = DataLoader()
loader.fit(val_x)
val_processed = loader.load_data()
print('data: ', val_processed[:10])

req_data = {'data': json.dumps(val_x.to_dict())}
response = requests.get('http://0.0.0.0:8000/predict', data=req_data)
api_predict = response.json()['prediction']
print('predict: ', api_predict[:10])

api_score = eval(metrics)(val_y, api_predict)
print('accuracy: ', api_score)

data:      Vmag    Plx  e_Plx    B-V       Amag
0   7.83   4.88   1.22  1.015  16.272099
1  10.69   0.73   1.28 -0.077  15.006614
2   7.02   9.23   0.85  0.518  16.846008
3   6.43   8.89   0.76  0.992  16.174510
4   8.47  11.01   0.94  0.549  18.678936
5   9.17   3.93   1.11  0.440  17.141964
6   5.57   8.49   0.73  0.035  15.214539
7   8.39  11.48   0.89  0.828  18.689709
8   9.02   4.42   2.56  0.579  17.247112
9   8.93   3.16   1.44  1.077  16.428436
predict:  [0, 1, 1, 0, 1, 1, 1, 1, 1, 0]
accuracy:  0.8686131386861314
